In [1]:
import pandas as pd
import pyranges as pr
import cerberus

In [2]:
from utils import *

In [3]:
iq_struct = 'IsoID_TranscriptID_TSS_Intron_PolyA'
ofile = 'scisoseq.gtf'
gene_meta = '/Users/fairliereese/mortazavi_lab/data/mousewg/refs/gencode_vM21_gene_metadata.tsv'

In [4]:
isoquant_struct_to_gtf(iq_struct, 
                       gene_meta,
                       ofile)

In [3]:
df = pd.read_csv('IsoID_TranscriptID_TSS_Intron_PolyA', sep='\t', header=None,
                names=['transcript_id', 'original_transcript_id', 'tss', 'ic', 'tes'])


In [4]:
df.head()

,transcript_id,original_transcript_id,tss,ic,tes
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107270_108107306_-
1,ENSMUSG00000000001.4_1,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107365_108107436_-
2,ENSMUSG00000000001.4_2,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108108143_108108180_-
3,ENSMUSG00000000001.4_3,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107622_108107622_-
4,ENSMUSG00000000001.4_4,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108108709_108108784_-


In [5]:
# pull chromosome and strand out from tss
df[['Chromosome', 'Strand']] = df.tss.str.split('_', expand=True)[[0,3]]

In [6]:
df.loc[df.Strand == '-'].head()

,transcript_id,original_transcript_id,tss,ic,tes,Chromosome,Strand
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107270_108107306_-,chr3,-
1,ENSMUSG00000000001.4_1,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107365_108107436_-,chr3,-
2,ENSMUSG00000000001.4_2,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108108143_108108180_-,chr3,-
3,ENSMUSG00000000001.4_3,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108107622_108107622_-,chr3,-
4,ENSMUSG00000000001.4_4,ENSMUST00000000001.4,chr3_108146081_108146150_-,;%;chr3_108109317_108109402_-;%;chr3_108109613...,chr3_108108709_108108784_-,chr3,-


In [7]:
df['coords'] = df.ic.str.split(';%;')
df.drop('ic', axis=1, inplace=True)

In [8]:
df = df.explode('coords', ignore_index=True)

In [9]:
# remove weird blank entries 
df = df.loc[df.coords != '']

In [10]:
# get start and stop of each intron
df[['Start', 'End']] = df.coords.str.split('_', expand=True)[[1,2]]
df[['Start', 'End']] = df[['Start', 'End']].astype(int)
df.drop('coords', axis=1, inplace=True)

In [11]:
print(len(df.loc[df.transcript_id=='ENSMUSG00000000056.7_0'].index))
df.loc[df.transcript_id=='ENSMUSG00000000056.7_0']

10


,transcript_id,original_transcript_id,tss,tes,Chromosome,Strand,Start,End
183,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121237352,121238418
184,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121238518,121242500
185,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121242645,121244593
186,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121244727,121244935
187,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121245071,121245337
188,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121245457,121246925
189,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121247056,121249128
190,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121249193,121250292
191,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121250431,121250759
192,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121250918,121252680


In [12]:
# increment / decrement coords to turn to exons
df.Start = df.Start-1
df.End = df.End+1

In [13]:
df.loc[df.transcript_id=='ENSMUSG00000000056.7_0']
df.loc[df.transcript_id=='ENSMUSG00000000001.4_0']

,transcript_id,original_transcript_id,tss,tes,Chromosome,Strand,Start,End
1,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108109316,108109403
2,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108109612,108111935
3,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108112088,108112473
4,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108112602,108115763
5,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108115891,108118301
6,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108118458,108123542
7,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108123683,108123795
8,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108123837,108145888


In [14]:
# melt to get coords individually
df = df.melt(id_vars=['transcript_id', 'original_transcript_id',
                 'tss', 'tes', 'Chromosome', 'Strand'],
                 value_vars=['Start', 'End'],
                 value_name='Coord')
df.drop('variable', axis=1, inplace=True)
df.Coord = df.Coord.astype(int)

fwd, rev = cerberus.get_stranded_gtf_dfs(df)
fwd.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, True], inplace=True)
rev.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, False], inplace=True)

        
        


In [15]:
fwd.loc[fwd.transcript_id=='ENSMUSG00000000056.7_0']

,transcript_id,original_transcript_id,tss,tes,Chromosome,Strand,Coord
159,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121237351
636555,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121238419
160,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121238517
636556,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121242501
161,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121242644
636557,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121244594
162,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121244726
636558,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121244936
163,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121245070
636559,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11_121237217_121237289_+,chr11_121254530_121254583_+,chr11,+,121245338


In [16]:
# get start / end coords of tss / tes regions
df[['tss_Start', 'tss_End']] = df.tss.str.split('_', expand=True)[[1,2]].astype(int)
df[['tes_Start', 'tes_End']] = df.tes.str.split('_', expand=True)[[1,2]].astype(int)

fwd, rev = cerberus.get_stranded_gtf_dfs(df)

# loop through strands and ends 
ends = pd.DataFrame()
for mode in ['tss', 'tes']:
    for strand, temp in zip(['+', '-'], [fwd, rev]):
        end_cols = ['{}_Start'.format(mode),
                    '{}_End'.format(mode)]
        keep_cols = ['transcript_id', 'original_transcript_id',
                     'Chromosome', 'Strand']
        keep_cols += end_cols
        temp = temp[keep_cols]
        old_end, new_end, func = cerberus.get_update_ends_settings(strand, mode)
        temp['Coord'] = temp[end_cols].apply(func, axis=1)
        # print(len(temp.loc[temp['Coord'].isnull()].index))
        temp.drop(end_cols, axis=1, inplace=True)
        temp.drop_duplicates(inplace=True)
        ends = pd.concat([ends, temp])

In [17]:
# ends.loc[ends.transcript_id=='ENSMUSG00000000056.7_0']
ends.loc[ends.transcript_id=='ENSMUSG00000000001.4_0']

,transcript_id,original_transcript_id,Chromosome,Strand,Coord
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108146150
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108107270


In [18]:
# df.loc[df.transcript_id=='ENSMUSG00000000056.7_0']
df.loc[df.transcript_id=='ENSMUSG00000000001.4_0']

,transcript_id,original_transcript_id,tss,tes,Chromosome,Strand,Coord,tss_Start,tss_End,tes_Start,tes_End
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108109316,108146081,108146150,108107270,108107306
1,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108109612,108146081,108146150,108107270,108107306
2,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108112088,108146081,108146150,108107270,108107306
3,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108112602,108146081,108146150,108107270,108107306
4,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108115891,108146081,108146150,108107270,108107306
5,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108118458,108146081,108146150,108107270,108107306
6,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108123683,108146081,108146150,108107270,108107306
7,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108123837,108146081,108146150,108107270,108107306
636396,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108109403,108146081,108146150,108107270,108107306
636397,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3_108146081_108146150_-,chr3_108107270_108107306_-,chr3,-,108111935,108146081,108146150,108107270,108107306


In [19]:
121237217 < 121237289

True

In [20]:
108146150.0>108107270

True

In [21]:
108107270.0<108107306

True

In [22]:
# drop unnecessary columns
df.drop(['tss', 'tes', 'tss_Start', 'tes_Start', 'tss_End', 'tes_End'], 
        axis=1, inplace=True)

In [23]:
# concatenate and then sort based on strand
df = pd.concat([df, ends])

In [24]:
fwd, rev = cerberus.get_stranded_gtf_dfs(df)
fwd.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, True], inplace=True)
rev.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                ascending=[True, True, False], inplace=True)
df = pd.concat([fwd, rev])

In [25]:
df.loc[df.transcript_id=='ENSMUSG00000000056.7_0']
df.loc[df.transcript_id=='ENSMUSG00000000001.4_0']

,transcript_id,original_transcript_id,Chromosome,Strand,Coord
0,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108146150
636403,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108145888
7,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108123837
636402,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108123795
6,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108123683
636401,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108123542
5,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108118458
636400,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108118301
4,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108115891
636399,ENSMUSG00000000001.4_0,ENSMUST00000000001.4,chr3,-,108115763


In [26]:
# add gene id
df['gene_id'] = df.transcript_id.str.split('_', expand=True)[0]

In [27]:
df.head()

,transcript_id,original_transcript_id,Chromosome,Strand,Coord,gene_id
26947,ENSMUSG00000003135.15_0,ENSMUST00000003219.13,chr1,+,39535748,ENSMUSG00000003135.15
26947,ENSMUSG00000003135.15_0,ENSMUST00000003219.13,chr1,+,39536337,ENSMUSG00000003135.15
663343,ENSMUSG00000003135.15_0,ENSMUST00000003219.13,chr1,+,39537416,ENSMUSG00000003135.15
26948,ENSMUSG00000003135.15_0,ENSMUST00000003219.13,chr1,+,39537580,ENSMUSG00000003135.15
663344,ENSMUSG00000003135.15_0,ENSMUST00000003219.13,chr1,+,39538271,ENSMUSG00000003135.15


In [28]:
# add gene name

In [29]:
gene_info = pd.read_csv('/Users/fairliereese/mortazavi_lab/data/mousewg/refs/gencode_vM21_gene_metadata.tsv', sep='\t')

In [30]:
gene_info.head()

,gid,gname,length,biotype,biotype_category,tf
0,ENSMUSG00000064842.1,Gm26206,109,snRNA,other,False
1,ENSMUSG00000051951.5,Xkr4,465597,protein_coding,protein_coding,False
2,ENSMUSG00000102851.1,Gm18956,479,processed_pseudogene,pseudogene,False
3,ENSMUSG00000103377.1,Gm37180,2818,TEC,other,False
4,ENSMUSG00000104017.1,Gm37363,2232,TEC,other,False


In [31]:
gene_info = gene_info[['gid', 'gname']]
gene_info.rename({'gid': 'gene_id', 'gname': 'gene_name'}, axis=1, inplace=True)

In [32]:
df = df.merge(gene_info, how='left', on='gene_id')

In [33]:
fwd, rev = cerberus.get_stranded_gtf_dfs(df)
df = pd.DataFrame()
for strand, temp in zip(['+', '-'], [fwd, rev]):
    if strand == '+':
        start_ind = 0
        end_ind = 1
    elif strand == '-':
        start_ind = 1
        end_ind = 0
        
    exons = pd.DataFrame({'Start':temp['Coord'].iloc[start_ind::2].values,
                          'Start_tid':temp['transcript_id'].iloc[start_ind::2].values,
                          'End':temp['Coord'].iloc[end_ind::2].values,
                          'End_tid':temp['transcript_id'].iloc[end_ind::2].values})
    temp = temp.iloc[::2]
    temp.drop('Coord', axis=1, inplace=True)

    print(exons.loc[exons.End.isnull()])
    print(exons.loc[exons.Start.isnull()])

    print(exons.loc[exons.Start_tid != exons.End_tid])

    exons.drop(['Start_tid','End_tid'], axis=1, inplace=True)

    print(len(temp.index))
    print(len(exons.index))
    # temp[['Start', 'End']] = exons[['Start', 'End']]
    temp['Start'] = exons['Start'].tolist()
    temp['End'] = exons['End'].tolist()
    # print(len(df.index))

    print(exons.loc[exons.End.isnull()])
    print(exons.loc[exons.Start.isnull()])

    print(len(temp.loc[temp.Start.isnull()].index))
    print(len(temp.loc[temp.End.isnull()].index))
    
    df = pd.concat([df, temp])

Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
383100
383100
Empty DataFrame
Columns: [Start, End]
Index: []
Empty DataFrame
Columns: [Start, End]
Index: []
0
0
Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
Empty DataFrame
Columns: [Start, Start_tid, End, End_tid]
Index: []
349389
349389
Empty DataFrame
Columns: [Start, End]
Index: []
Empty DataFrame
Columns: [Start, End]
Index: []
0
0


In [34]:
# add exon feature tag 
df['Feature'] = 'exon'

In [35]:
def make_hier_entry(df, how='t'):
    """
    kind {'g','t'}
    """
    agg_dict = {'min_coord': 'min', 'max_coord': 'max'}
    t_df = df.copy(deep=True)
    t_df['min_coord'] = t_df[['Start', 'End']].min(axis=1)
    t_df['max_coord'] = t_df[['Start', 'End']].max(axis=1)
    if how == 't':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id', 'transcript_id', 'transcript_name',
                   'tss_id', 'tes_id',
                   'new_transcript_id', 'original_transcript_id',
                   'original_transcript_name', 'ag1', 'ag2']
        gb_cols = list(set(gb_cols)&(set(t_df.columns)))
    elif how == 'g':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id']

    cols = gb_cols + ['min_coord', 'max_coord']
    t_df = t_df[cols]
    t_df = t_df.groupby(gb_cols).agg(agg_dict).reset_index()
    t_df.rename({'min_coord': 'Start', 'max_coord': 'End'}, axis=1, inplace=True)
    if how == 't':
        t_df['Feature'] = 'transcript'
    elif how == 'g':
        t_df['Feature'] = 'gene'

    return t_df

def make_gtf_from_exons(df):
    
    # make transcript entries
    t_df = make_hier_entry(df, how='t')
    
    # make gene entries
    g_df = make_hier_entry(df, how='g')

    # concat everything and sort by gene id, transcript id, feature rank (gene =0, t =1, exon=2), then start coords
    df = pd.concat([df, t_df, g_df])
    df = cerberus.sort_gtf(df)
    return df

In [36]:
df.loc[df.Start.isnull()]

,transcript_id,original_transcript_id,Chromosome,Strand,gene_id,gene_name,Start,End,Feature


In [37]:
df = make_gtf_from_exons(df)

In [38]:
df.head()

,transcript_id,original_transcript_id,Chromosome,Strand,gene_id,gene_name,Start,End,Feature
0,NaN,NaN,chr11,+,ENSMUSG00000000056.7,Narf,121237217,121255865,gene
1,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121237217,121254583,transcript
2,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121237217,121237351,exon
3,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121238419,121238517,exon
4,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121242501,121242644,exon


In [39]:
df.loc[df.End.isnull()]

,transcript_id,original_transcript_id,Chromosome,Strand,gene_id,gene_name,Start,End,Feature


In [40]:
df.loc[df.transcript_id=='ENSMUSG00000000056.7_0']
# df.loc[df.transcript_id=='ENSMUSG00000000001.4_0']

,transcript_id,original_transcript_id,Chromosome,Strand,gene_id,gene_name,Start,End,Feature
1,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121237217,121254583,transcript
2,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121237217,121237351,exon
3,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121238419,121238517,exon
4,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121242501,121242644,exon
5,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121244594,121244726,exon
6,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121244936,121245070,exon
7,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121245338,121245456,exon
8,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121246926,121247055,exon
9,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121249129,121249192,exon
10,ENSMUSG00000000056.7_0,ENSMUST00000103015.3,chr11,+,ENSMUSG00000000056.7,Narf,121250293,121250430,exon


In [41]:
# decrement the start coordinate to convert bed style coordinates
# to gtf style coordinates
df['Start'] = df['Start']-1

# save example gtfs 
tids = ['ENSMUSG00000000056.7_0', 'ENSMUSG00000000001.4_0']
for t in tids:
    o = '{}.gtf'.format(t)
    temp = df.loc[df.transcript_id == t]
    temp = pr.PyRanges(temp)
    temp.to_gtf(o)

In [42]:
df = pr.PyRanges(df)

In [43]:
df.to_gtf(ofile)